In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Preprocesado de datos

In [2]:

# Cargar datos (asumiendo CSV, si el formato es diferente avísame)
df = pd.read_csv("../0.data/casas.trn.txt", sep='\s+' , header=None)

# Separar características (X) y etiquetas (y)
X = df.iloc[:, :-1].values  # Todas las columnas menos la última
y = df.iloc[:, -1].values   # Última columna (precio de la vivienda)

# Normalizar los datos (media 0, varianza 1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convertir a tensores de PyTorch
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y.reshape(-1, 1), dtype=torch.float32)

# Dividir en conjuntos de entrenamiento (70%), validación (15%) y prueba (15%)
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


# Estadistica

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
import ipywidgets as widgets
from ipywidgets import interact

def plot_normal_distribution(mean=0, median=0, std_dev=1):
    """
    Grafica una distribución normal y permite modificar la media, mediana y desviación estándar con deslizadores.
    
    Parámetros:
    - mean: Media (μ) de la distribución normal
    - median: Mediana (se puede modificar para ver efectos de sesgo)
    - std_dev: Desviación estándar (σ), controla la dispersión
    """
    # Rango de valores para la distribución
    x = np.linspace(mean - 4*std_dev, mean + 4*std_dev, 1000)
    
    # Distribución normal con la media y desviación estándar dadas
    y = norm.pdf(x, mean, std_dev)

    # Crear la figura
    plt.figure(figsize=(8, 5))
    
    # Graficar la distribución
    sns.lineplot(x=x, y=y, label="Distribución Normal", color="blue")
    
    # Línea para la media
    plt.axvline(mean, color='red', linestyle='--', label=f"Media (μ): {mean:.2f}")
    
    # Línea para la mediana (puede ser diferente si hay sesgo)
    plt.axvline(median, color='green', linestyle=':', label=f"Mediana: {median:.2f}")
    
    # Configuración del gráfico
    plt.title("Distribución Normal con Parámetros Ajustables")
    plt.xlabel("Valores")
    plt.ylabel("Densidad de Probabilidad")
    plt.legend()
    plt.grid()
    plt.show()

# Interfaz interactiva con deslizadores
interact(plot_normal_distribution,
         mean=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description="Media (μ)"),
         median=widgets.FloatSlider(min=-10, max=10, step=0.1, value=0, description="Mediana"),
         std_dev=widgets.FloatSlider(min=0.1, max=5, step=0.1, value=1, description="Desviación (σ)"));


interactive(children=(FloatSlider(value=0.0, description='Media (μ)', max=10.0, min=-10.0), FloatSlider(value=…

# Modelo

In [4]:
import torch.nn as nn  # Importamos el módulo de redes neuronales de PyTorch

# Definimos la clase del perceptrón, heredando de nn.Module
class Perceptron(nn.Module):
    def __init__(self, input_size, hidden_size):
        """
        Constructor del perceptrón.
        
        Parámetros:
        - input_size: Número de entradas (cantidad de características de cada muestra).
        - hidden_size: Número de neuronas en la capa oculta.
        """
        
        super().__init__()  # Inicializa nn.Module para registrar parámetros
        
        # Definimos la capa oculta (fully connected)
        self.hidden = nn.Linear(input_size, hidden_size)  
        self.f_hidden = nn.Tanh()

        # Definimos la capa de salida con 1 neurona (para regresión)
        self.output = nn.Linear(hidden_size, 1)
        # Esto es la función identidad, realmente no haria falta ponerla
        # en forward simplemnte no le aplicamos ninguna función solo la capa
        self.f_output = lambda x: x

    def forward(self, x):
        """
        Define el flujo de datos a través de la red neuronal.
        
        Parámetro:
        - x: Tensor de entrada (muestra o batch de datos)
        
        Retorna:
        - La predicción del modelo (salida de la capa final)
        """
        x = self.f_hidden(self.hidden(x))  # Aplicamos la capa oculta + activación Tanh
        x = self.f_output(self.output(x))  # Aplicamos la capa de salida sin activación
        return x  # Devolvemos la salida final del modelo

# Crear el modelo con los tamaños adecuados
input_size = X.shape[1]  # Número de características de entrada (columnas del dataset)
hidden_size = 3  # Número de neuronas en la capa oculta (valor arbitrario)

model = Perceptron(input_size, hidden_size)  # Instanciamos el modelo


In [5]:
# Argumento para saer en que estado esta el modelo train o eval
model.training
# model.eval()
# model.training

True

In [6]:
import torch.optim as optim

criterion = nn.MSELoss()  # Error cuadrático medio
optimizer = optim.SGD(model.parameters(), lr=0.002)  # Descenso de gradiente


In [7]:
epochs = 10000  # Número de iteraciones

for epoch in range(epochs):
    model.train()  # Modo entrenamiento
    optimizer.zero_grad()  # Resetear gradientes
    
    output = model(X_train)  # Forward pass
    loss = criterion(output, y_train)  # Calcular pérdida
    loss.backward()  # Backpropagation
    optimizer.step()  # Actualizar pesos

    if epoch % 100 == 0:
        print(f"Época {epoch}, Pérdida: {loss.item():.4f}")


Época 0, Pérdida: 592.8000
Época 100, Pérdida: 111.5889
Época 200, Pérdida: 57.9070
Época 300, Pérdida: 47.6761
Época 400, Pérdida: 37.5689
Época 500, Pérdida: 30.4651
Época 600, Pérdida: 25.6231
Época 700, Pérdida: 22.2665
Época 800, Pérdida: 19.7878
Época 900, Pérdida: 17.9129
Época 1000, Pérdida: 16.5206
Época 1100, Pérdida: 15.4988
Época 1200, Pérdida: 14.7307
Época 1300, Pérdida: 14.1244
Época 1400, Pérdida: 13.6129
Época 1500, Pérdida: 13.1332
Época 1600, Pérdida: 12.5480
Época 1700, Pérdida: 11.9904
Época 1800, Pérdida: 11.6590
Época 1900, Pérdida: 11.3925
Época 2000, Pérdida: 11.1728
Época 2100, Pérdida: 10.9877
Época 2200, Pérdida: 10.8286
Época 2300, Pérdida: 10.6894
Época 2400, Pérdida: 10.5658
Época 2500, Pérdida: 10.4550
Época 2600, Pérdida: 10.3550
Época 2700, Pérdida: 10.2642
Época 2800, Pérdida: 10.1815
Época 2900, Pérdida: 10.1058
Época 3000, Pérdida: 10.0365
Época 3100, Pérdida: 9.9729
Época 3200, Pérdida: 9.9142
Época 3300, Pérdida: 9.8601
Época 3400, Pérdida: 9.8100

In [8]:
model.eval()  # Cambiar a modo evaluación
with torch.no_grad():
    y_pred = model(X_test)
    test_loss = criterion(y_pred, y_test)

print(f"Error cuadrático medio en test: {test_loss.item():.4f}")


Error cuadrático medio en test: 18.7337
